In [181]:
from nba_api.stats.endpoints import leagueleaders
import pandas as pd
import helper_functions as hf  # get true shooting percentage, etc.
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, LabelSet

In [3]:
### for season in range
### get_league_leaders(season)

In [61]:
#merge all to dataframe
df = pd.concat([pd.read_csv("data/league_leaders/{}.csv".format(year)) for year in range(1980, 2024)])
store_df = df.copy()

In [62]:
df["TS%"] = df.apply(hf.calculate_trueshooting,axis=1)
# df = df.dropna(subset=["TS%"])
# df = df[df["FGA"] > 100]

In [103]:
tsp_by_year = df.groupby('Season').apply(
    lambda group: group['PTS'].sum() / (2 * (group['FGA'].sum() + 0.44 * group['FTA'].sum()))
)

# merge the series with df and name the column lgts, ValueError: Cannot merge a Series without a name
tsp_by_year.name = 'lgts'
df = df.merge(tsp_by_year, left_on='Season', right_on='Season', suffixes=('', '_lgts'))


In [105]:
df["FG Add"] = df.apply(lambda row: hf.calculate_fgadd(row, row["lgts"]), axis=1)

In [222]:
league_average = df.groupby('Season').agg(TS = ('lgts', 'mean'), FGAdd = ('FG Add', 'max'), FG3A = ('FG3A', 'sum'))

# get the max FGAdd for each season, and the corresponding player
max_fgadd = df.groupby('Season').agg(FGAdd = ('FG Add', 'max'))
max_fgadd = max_fgadd.merge(df, left_on='FGAdd', right_on='FG Add', suffixes=('', '_player'))

In [223]:
max_fgadd.index = max_fgadd['Season']
max_fgadd

,FGAdd,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,...,PF,PTS,EFF,AST_TOV,STL_TOV,Season,TS%,TS%_league_avg,lgts,FG Add
Season,,,,,,,,,,,,,,,,,,,,,
1980,353.344514,600014,28,Artis Gilmore,1610612741,CHI,82,2832,547,816,...,295,1469,2052,0.73,0.20,1980,0.699471,0.509468,0.536187,353.344514
1981,365.131473,76504,3,Adrian Dantley,1610612762,UTH,81,3222,904,1586,...,252,2457,2253,1.08,0.32,1981,0.631321,0.518828,0.539809,365.131473
1982,311.452917,600014,33,Artis Gilmore,1610612759,SAN,82,2797,556,888,...,273,1479,2106,0.50,0.16,1982,0.668481,0.508859,0.531397,311.452917
1983,417.226863,76504,1,Adrian Dantley,1610612762,UTH,79,2984,802,1438,...,201,2418,2209,1.18,0.23,1983,0.652019,0.521181,0.542855,417.226863
1984,321.195169,600014,30,Artis Gilmore,1610612759,SAN,81,2756,532,854,...,306,1548,2013,0.54,0.17,1984,0.679997,0.514429,0.543551,321.195169
1985,322.580260,76504,3,Adrian Dantley,1610612762,UTH,76,2744,818,1453,...,206,2267,1962,1.14,0.28,1985,0.628591,0.513915,0.541454,322.580260
1986,366.812008,1450,8,Kevin McHale,1610612738,BOS,77,3060,790,1307,...,240,2008,2381,1.00,0.19,1986,0.655233,0.510894,0.537890,366.812008
1987,448.049091,787,5,Charles Barkley,1610612755,PHL,80,3170,753,1283,...,278,2264,2601,0.84,0.33,1987,0.665319,0.511080,0.537924,448.049091
1988,373.139572,787,9,Charles Barkley,1610612755,PHL,79,3088,700,1208,...,262,2037,2582,1.28,0.50,1988,0.653069,0.507277,0.537007,373.139572


In [224]:
league_average = league_average.merge(max_fgadd["PLAYER"], left_index=True,right_index=True)

In [231]:
league_average

,TS,FGAdd,FG3A,PLAYER
Season,,,,
1980,0.536187,353.344514,3483,Artis Gilmore
1981,0.539809,365.131473,4043,Adrian Dantley
1982,0.531397,311.452917,4248,Artis Gilmore
1983,0.542855,417.226863,4483,Adrian Dantley
1984,0.543551,321.195169,5849,Artis Gilmore
1985,0.541454,322.580260,5983,Adrian Dantley
1986,0.537890,366.812008,8913,Kevin McHale
1987,0.537924,448.049091,9421,Charles Barkley
1988,0.537007,373.139572,13431,Charles Barkley


In [233]:
# turn league average into a dict with index as a column

temp = league_average.reset_index()
league_source = temp.to_dict(orient='list')

In [289]:
source = ColumnDataSource(data = dict(Season = list(league_source["Season"]),  FGAdd = list(league_source["FGAdd"]), Player = list(league_source["PLAYER"]) , TS = list(league_source["TS"]), FG3A = list(league_source["FG3A"])))

In [290]:
source

ColumnDataSource(id='40080', ...)

In [310]:

output_notebook()

# Create a figure
p = figure(title="League Average by Season", x_axis_label="Season", y_axis_label="League Average", x_range=(1980, 2030), y_range=(200, 500) )
# Add a line plot
p.scatter(x ="Season", y ="FGAdd", line_width=2, color="purple", source=source)


# add a second y-axis

p.extra_y_ranges = {"FG3A": Range1d(start=0, end=90000), "TS": Range1d(start=.5, end=.6)}

p.add_layout(LinearAxis(y_range_name="FG3A", axis_label="3-Point Attempts"), 'right')
p.line("Season", "FG3A", line_width=2, y_range_name="FG3A", color="red", source=source)


p.add_layout(LinearAxis(y_range_name="TS", axis_label="League AVG ts"), 'right')
p.line("Season", "TS", line_width=2, y_range_name = "TS", source=source)

labels = LabelSet(x="Season" , y="FGAdd", text='Player',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')

p.add_layout(labels)

# Show the plot
show(p)

Loading BokehJS ...

In [36]:
df = df.dropna(subset=["TS%"])
# drop players with less than 100 shots
df = df[df["FGA"] > 100]

In [37]:
df.sort_values(by="TS%", ascending=False).head(10)

,PLAYER_ID,RANK,PLAYER,TEAM_ID,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,AST,STL,BLK,TOV,PF,PTS,EFF,AST_TOV,STL_TOV,TS%
364,1630579,365,Jericho Sims,1610612752,NYK,52,812,83,107,0.776,...,25,15,28,27,96,178,433,0.93,0.56,0.780428
183,1629057,183,Robert Williams III,1610612738,BOS,61,1804,271,368,0.736,...,121,56,134,63,137,607,1321,1.92,0.89,0.744603
308,1629057,309,Robert Williams III,1610612738,BOS,35,824,127,170,0.747,...,50,22,48,34,68,279,598,1.47,0.65,0.741863
150,1629655,151,Daniel Gafford,1610612764,WAS,78,1604,289,395,0.732,...,84,33,99,85,187,705,1103,0.99,0.39,0.738560
226,203939,227,Dwight Powell,1610612742,DAL,76,1458,197,269,0.732,...,69,48,26,65,209,506,770,1.06,0.74,0.737781
221,201599,222,DeAndre Jordan,1610612751,BKN,57,1246,190,249,0.763,...,93,17,65,85,116,426,838,1.09,0.20,0.735802
73,203497,74,Rudy Gobert,1610612762,UTA,66,2120,362,508,0.713,...,72,45,137,119,176,1027,1848,0.60,0.38,0.732358
350,201599,350,DeAndre Jordan,1610612743,DEN,39,586,88,115,0.765,...,34,12,23,48,71,199,369,0.71,0.25,0.730973
280,1628436,280,Luke Kornet,1610612738,BOS,59,904,127,183,0.694,...,58,23,57,20,73,298,594,2.90,1.15,0.728393
220,1641726,221,Dereck Lively II,1610612742,DAL,55,1294,221,296,0.747,...,60,36,77,50,144,483,869,1.20,0.72,0.728199
